In [71]:
import numpy as np

import gc
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import *
from tensorflow.python.keras.layers import Layer
from tensorflow.keras import regularizers

from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

from tensorflow.keras import optimizers,initializers
from tensorflow.python.keras.initializers import glorot_normal
import joblib
import json

from tensorflow.keras import layers, Model
import utils
from utils import *
import importlib
importlib.reload(utils)

from features import *
from tensorflow.keras.callbacks import LambdaCallback

import models
from models import *
importlib.reload(models)
from tensorflow.keras.optimizers import Adam

import layers
from layers import *
importlib.reload(layers)

import model_config
from model_config import *
importlib.reload(model_config)

import features
from features import *
importlib.reload(features)

<module 'features' from '/Users/bytedance/Desktop/wechat_MTL/features.py'>

In [2]:
train = joblib.load('./data_and_feature/train.txt')
val = joblib.load('./data_and_feature/val.txt')
test = joblib.load('./data_and_feature/test.txt')
encoder = joblib.load('./data_and_feature/encoder.txt')
encoder_description = joblib.load('./data_and_feature/encoder_description.txt')

train_num = len(train)

In [3]:
encoder_description = joblib.load('./data_and_feature/encoder_description.txt')

In [39]:
embedding_feat_dict=read_json_file('/Users/bytedance/Desktop/wechat_MTL/config/embedding_feat_dict.json')

In [5]:
# 构建输入数据
train_model_input = {name: train[name] if name not in varlen_features else np.stack(train[name]) for name in feature_names } #训练模型的输入，字典类型。名称和具体值
val_model_input = {name: val[name] if name not in varlen_features else np.stack(val[name]) for name in feature_names }
test_model_input = {name: test[name] if name not in varlen_features else np.stack(test[name]) for name in feature_names}

train_labels = [train[y].values for y in target]
val_labels = [val[y].values for y in target]

In [40]:
sparse_features = embedding_feat_dict['sparse']

sequence_features = embedding_feat_dict['sequence']

dense_features = embedding_feat_dict['dense']

task_names = target

In [72]:
model = MMoE_model(dense_features, sparse_features, varlen_features, task_names, embedding_feat_dict)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss={task: 'binary_crossentropy' for task in target},
    metrics={task: ['AUC'] for task in target}
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ description         │ (None, 622)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_description     │ (None, 622, 6)    │    278,934 │ description[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_8 (GRU)         │ (None, 622, 64)   │     13,824 │ emb_description[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 622, 1)    │         65 │ gru_8[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ videoplayseconds    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ userid (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feedid (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ authorid            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bgm_song_id         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bgm_singer_id       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ manual_tag_list     │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ manual_keyword_list │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_3 (Softmax) │ (None, 622, 1)    │          0 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1)         │          4 │ videoplayseconds… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_userid          │ (None, 1, 6)      │    120,006 │ userid[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_feedid          │ (None, 1, 6)      │    595,032 │ feedid[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_authorid        │ (None, 1, 6)      │    111,744 │ authorid[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_bgm_song_id     │ (None, 1, 6)      │    142,440 │ bgm_song_id[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,553,153 (5.92 MB)

 Trainable params: 1,553,151 (5.92 MB)

 Non-trainable params: 2 (8.00 B)

In [74]:
model.fit(
    train_model_input,
    train_labels,
    batch_size=256,
    epochs=1,
    validation_split=0.1,
    verbose=1)

 1122/23586 ━━━━━━━━━━━━━━━━━━━━ 1:35:27 255ms/step - click_avatar_AUC: 0.5426 - click_avatar_loss: 0.0478 - forward_AUC: 0.5055 - forward_loss: 0.0261 - like_AUC: 0.5631 - like_loss: 0.1270 - loss: 0.3479 - read_comment_AUC: 0.6402 - read_comment_loss: 0.1471

KeyboardInterrupt: 